# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from numpy import dot
from numpy.linalg import norm

In [ ]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [ ]:
config = pd.read_csv('config.csv').iloc[0]
config
user = config['id']
k_value = config['k']

### Read the Data

In [ ]:
#with open("/content/Behance_subsampled.txt",'r') as inFile:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()
inFile.close()

In [ ]:
print(type(appreciate_data))
print(len(appreciate_data))
print(appreciate_data[0])

<class 'list'>
1000000
276633 01588231 1307583271



In [ ]:
print(appreciate_data[0:4])

['276633 01588231 1307583271\n', '1238354 01529213 1307583273\n', '165550 00485000 1307583337\n', '2173258 00776972 1307583340\n']


In [ ]:
user_likes = dict()
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)
#print(user_likes)
print(len(user_likes))

63497


In [ ]:
#creating row,column,data for sparse matrix
row = []
column = []
data = []
for like in appreciate_data:
  like = like.strip()
  row.append(int(like.split()[0]))
  column.append(int(like.split()[1]))
  data.append(1)

In [ ]:
#user and item mapping with index
#(user id, item id, unix timestamp)
user_tags = {}
index = 0
for user in row:
  if user not in user_tags.keys():
    user_tags[user] = index
    index += 1
print(len(user_tags))

item_tags = {}
index = 0
for item in column:
  if item not in item_tags.keys():
    item_tags[item] = index
    index += 1
print(len(item_tags))

63497
178788


In [ ]:
users = []
for user in row:
  users.append(user_tags[user])
del row
items = []
for item in column:
  items.append(item_tags[item])
del column

In [ ]:
#print(data)
print(len(items))
print(len(users))
print(len(data))
print(len(user_tags))
print(len(item_tags))

1000000
1000000
1000000
63497
178788


In [ ]:
users = np.array(users, dtype = np.float32)
items = np.array(items,dtype = np.float32)
data = np.array(data,dtype = np.float32)
matrix = csr_matrix((data, (users, items)), shape=(len(user_tags), len(item_tags)))

In [ ]:
#print(matrix.toarray())
print(matrix.shape)

(63497, 178788)


In [ ]:
svd = TruncatedSVD(n_components=4000)
svd_matrix = svd.fit_transform(matrix)
print(svd.explained_variance_ratio_.sum())


0.7264493


####Unable to run below cell due to hardware constraints to select better n_components

In [ ]:
def select_n_components(matrix,target_variance):
  """For selecting best number of n_components for capturing the variability of data well enough"""
  n_components_selected = 0
  while(True):
    n_components_selected += 200
    svd = TruncatedSVD(n_components = n_components_selected,random_state=23)
    svd.fit(matrix)
    if svd.explained_variance_ratio_.sum() >= target_variance :
      break
  return svd,n_components_selected

return_components = select_n_components(matrix,0.85)
print("Best Number for n_components:",return_components[1])
svd_matrix = return_components[0].fit_transform(matrix)

we can't compute for n_components greater than 4000 because of hardware limit.
If we increase n_components such that explained_variance_ratio.sum() gets to 0.85  to 0.90 we can see better results.

In [ ]:
print(svd_matrix.shape)

In [ ]:
# tsvd = TruncatedSVD(n_components=3000)
# X_tsvd = tsvd.fit(matrix)
# tsvd_var_ratios = tsvd.explained_variance_ratio_
# print(tsvd_var_ratios.sum())

In [ ]:
# tsvd = TruncatedSVD(n_components=matrix.shape[1]-1)
# X_tsvd = tsvd.fit(matrix)
# tsvd_var_ratios = tsvd.explained_variance_ratio_
# select_n_components(tsvd_var_ratios, 0.95)

In [ ]:
#print(svd_matrix)
print(type(svd_matrix))
print(svd_matrix.shape)

<class 'numpy.ndarray'>
(63497, 4000)


In [ ]:
print(matrix.shape)
print(type(matrix))

(63497, 178788)
<class 'scipy.sparse.csr.csr_matrix'>


### Use KNN after Collaborative Filtering to find nearest neighbors

In [ ]:
def cosine_distance(node,test_node):
  cos_sim = 1 - (dot(node,test_node)/(norm(test_node)*norm(node)))
  return cos_sim

In [ ]:
def find_max(l):
  max = -1
  max_key = 0
  for item in l.keys():
    if l[item] > max:
      max = l[item]
      max_key = item
  return max_key

##KNN Built-from-scratch Model

In [ ]:
def knneighbors(user,k_value,svd_matrix):
  k_neighbors = {}
  user_tag = user_tags[user]
  user_node = svd_matrix[user_tag]
  for itr in range(len(svd_matrix)):
    if itr == user_tag:
      continue
    else:
      node = svd_matrix[itr]
      distance = cosine_distance(user_node,node)
      if len(k_neighbors) < k_value:
        k_neighbors[itr] = distance
      else:
        max_dist = find_max(k_neighbors)
        #an iterator for max_distance..!
        if k_neighbors[max_dist] > distance :
          k_neighbors.pop(max_dist)
          k_neighbors[itr] = distance
  k_neighbors = {key:value for key,value in sorted(k_neighbors.items(),key = lambda item: item[1])}
  return list(k_neighbors.keys())


In [ ]:
neighbor_tags = knneighbors(user,k_value,svd_matrix)
neighbors = []
for j in neighbor_tags:
  for l in user_tags.keys():
    if user_tags[l] == j:
      neighbors.append(l)
#print(neighbor_tags)
print("Neighbors:",neighbors)

Neighbors: [1333087, 2059570, 3613632, 938435, 3629295]


In [ ]:
appreciate_data.clear()

In [ ]:
for item in neighbors:
  print(user_likes[item])

[1945173, 329834, 2037507, 1912005, 2062516]
[329834]
[329834]
[329834]
[329834]


### Output file to write all the lines to the file

In [ ]:
outFile = open("output.txt", 'w')
liked_items = set(user_likes[user])
for n_user in neighbors:
  for item_id in user_likes[n_user]:
    if item_id not in liked_items:
      outFile.write(str(item_id) + ' ' + str(n_user) + '\n')
      liked_items.add(item_id)
outFile.close()